# 리뷰 데이터 분류

과정은 아래와 같다.

1. 데이터는 제공

2. 형태소 분석(mecab ver)

    2-1. train set에 있는 단어로 사용자 사전 구축
    
    2-2. 형태소 분석을 한 데이터를 이용(형태소 분석 mecab이 vmware상에 존재해서 속도 측면 때문에 이렇게 진행)
    

3. 모델링

4. 훈련

5. 평가

6. 번외편(정확성이 아닌 다른 방법으로 테스트 해본 것)

    6-1. 머신러닝
    
    6-2. 카카오 형태소(khaiii)를 이용한 분석(개인 블로그에 설치 및 리뷰 올려놓았다. 해당 편 커널에 주소가 있다.)
    
    
<img src='./assets/0.PNG'>

먼저 위와 같이 사용자 사전을 구축한다.

<img src='./assets/1.PNG'>
<img src='./assets/1-2.PNG'>
<img src='./assets/1-3.PNG'>
그리고 위 처럼 형태소 분석을 진행하는 파이썬 프로그램을 구현

<img src='./assets/2.PNG'>

완료.

<img src='./assets/3.PNG'>

형태소 분석이 완료된 데이터


이제 위와 같이 완료된 데이터로 진행한다.

In [1]:
import os, json, glob, sys, numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import keras.backend.tensorflow_backend as K
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Flatten, Dropout, Input, Conv1D, MaxPooling1D, Activation, GlobalMaxPooling1D, GlobalMaxPool1D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

C:\Users\leesoojin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


헤더가 없으니 헤더를 None으로 해서 가지고 온다.


In [2]:
data = pd.read_csv('./data_after_tokenizer/ratings_train_after_prepro.txt', header=None)
data_test = pd.read_csv('./data_after_tokenizer/ratings_test_after_prepro.txt', header=None)

train에 15만개, test에 5만개 데이터 셋 확인

In [3]:
print(len(data.iloc[:, 0]))
print(data.shape)
print(data_test.shape)

150000
(150000, 3)
(50000, 3)


뭐 굳이 섞어줄 필요는 없지만 그냥 섞어준다.

In [4]:
df = data.sample(frac=1).reset_index(drop=True)
print(df.iloc[0:10,2])

0    0
1    1
2    0
3    1
4    1
5    1
6    1
7    0
8    0
9    1
Name: 2, dtype: int64


또한, 평가 데이터 중 평가 글이 없는 데이터가 있으므로 제거.

In [5]:
df = df.dropna(how='any')
data_test = data_test.dropna(how='any')

train, test로 나눈다.

In [6]:
X_train = df.iloc[:, 1].values
y_train = df.iloc[:, 2].values
X_test = data_test.iloc[:, 1].values
y_test = data_test.iloc[:, 2].values


In [7]:
print(X_train[:5])
print(X_test[:5])


['잔잔 흐름 속 피식피식 ' '아주 재밌 고요 저 만화 봐요 ' '시시 전개 시시 결말 촌 전 영화 '
 '영화 최고 한 번 같 다 다른 세 네 번 최고 배우 ' '쉴새없이 몰아치 는 액션 눈 즐겁 다 년 전 영화 라 믿 어려울 정도 ']
['굳 크 ' '뭐 이 평점 .. .. 나쁘 않 지만 10점 더더욱 잖아 '
 '지루 지 않 은데 완전 막장 돈 주 고 보 .. .. '
 '었 어도 별 개 줬 을 텐데 .. 왜 나와서 심기 불편 게 하 죠 ? ? ' '음악 주가 된 최고 음악 영화 ']


**원래 본인이 하던 스타일은 상위 10%~20% 단어만 사용하는데, 네티즌들이 사용하는 단어는 너무 다양해서 최대한 많은 단어를 사용**

그리고 최대 길이가 37이었나? 그래서 길이는 적당히 35로 지정.

그리고 Tokenizer를 통해 단어를 토큰화

In [8]:
max_word = 35000
max_len = 35

tok = Tokenizer(num_words = max_word)
tok.fit_on_texts(X_train)
print(len(tok.word_index))
#print(tok.word_index)

42251


텍스트 -> 숫자 변경

In [9]:
sequences = tok.texts_to_sequences(X_train)
print(len(sequences[0]))
print(sequences[0])

4
[278, 1095, 160, 9804]


시퀀스 메트릭스로 변경

In [10]:
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)
print(len(sequences_matrix))
print(sequences_matrix[0])

148915
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0  278 1095  160 9804]


In [11]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_test_matrix = sequence.pad_sequences(sequences_test, maxlen=max_len)

# 모델 구축

In [12]:
with K.tf_ops.device('device:GPU:0'):
    model = Sequential()
    #max_words를 50차원에. 즉 20000개의 단어를 50차원에다가. 문장의 길이는 max_len
    model.add(Embedding(max_word, 50, input_length=max_len))
    model.add(LSTM(64))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model_dir = './model'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_path = model_dir + "/review_lstm_soojin.model"
    
    checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 50)            1750000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 1,796,337
Trainable params: 1,796,337
Non-trainable params: 0
_________________________________________________________________


** 훈련 **

In [14]:
hist = model.fit(sequences_matrix, y_train, batch_size=128, epochs=15, validation_split=0.2, callbacks=[checkpoint, early_stopping])

Train on 119132 samples, validate on 29783 samples
Epoch 1/15
119132/119132 [==============================] - 61s 509us/step - loss: 0.3904 - acc: 0.8225 - val_loss: 0.3387 - val_acc: 0.8537

Epoch 00001: val_loss improved from inf to 0.33870, saving model to ./model/review_lstm_soojin.model
Epoch 2/15
119132/119132 [==============================] - 58s 489us/step - loss: 0.2915 - acc: 0.8775 - val_loss: 0.3407 - val_acc: 0.8548

Epoch 00002: val_loss did not improve from 0.33870
Epoch 3/15
119132/119132 [==============================] - 58s 491us/step - loss: 0.2368 - acc: 0.9021 - val_loss: 0.3784 - val_acc: 0.8535

Epoch 00003: val_loss did not improve from 0.33870
Epoch 4/15
119132/119132 [==============================] - 58s 489us/step - loss: 0.1957 - acc: 0.9209 - val_loss: 0.3961 - val_acc: 0.8499

Epoch 00004: val_loss did not improve from 0.33870


** 정확도 **

In [15]:
print("정확도 : %.4f" % (model.evaluate(sequences_test_matrix, y_test)[1]))

49631/49631 [==============================] - 36s 716us/step
정확도 : 0.8460


# 번외편

** 사이킷런 및 카카오 형태소 분석기로 했을 때의 성능 비교 **

** 카카오 형태소 분석기는 본인 블로그에 설치 방법 및 리뷰를 등록해놓았다. 참고**

**설치 및 간단 리뷰**
https://lsjsj92.tistory.com/408

**모델 비교 및 성능 평가**
https://lsjsj92.tistory.com/410


# 사이킷런

TfidfVectorizer와 logisticregression 이용.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.pipeline import make_pipeline

In [17]:
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {
    'logisticregression__C' : [0.001, 0.01, 0.1, 1, 10]
}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
print("최상의 교차 검증 점수 : %.2f" %(grid.best_score_))

최상의 교차 검증 점수 : 0.82


In [18]:
print(" 테스트 점수 : %.2f" %(grid.score(X_test, y_test)))

 테스트 점수 : 0.82


In [19]:
vector = grid.best_estimator_.named_steps["tfidfvectorizer"]

In [20]:
indices = np.argsort(vector.idf_)[::-1]
features = vector.get_feature_names()
top_n = 2
top_features = [features[i] for i in indices[:top_n]]

In [21]:
X_train_tf = vector.transform(X_train)
feature_names = np.array(vector.get_feature_names())
max_value = X_train_tf.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
print("tfidf가 가장 높은 특성 100 개 : \n ", feature_names[sorted_by_tfidf[-100:]])

tfidf가 가장 높은 특성 100 개 : 
  ['승부' '승리' '승려' '습니당' '습니다' '습격' '슬플' '슬프' '슬펐' '캐리' '슬퍼요' '슬퍼' '캐릭터'
 '스피드' '스포일러' '스페인' '시간대' '스릴러' '친절' '친일파' '씨발' '씁쓸' '쓴다' '쓰레기통' '충무로'
 '충분' '충분히' '씨엔블루' '쓰레기' '쓰래기' '쏘우' '쏘리' '썸머' '썰전' '썰렁' '취미' '쓰레' '아가'
 '아기자기' '아까' '아래' '아라' '아들' '아드레날린' '아동' '아담' '아니하' '아니야' '아니' '아놔' '아나'
 '아깝' '출연료' '아까웠' '출연진' '아까워' '아까운' '써요' '신데렐라' '쌍둥이' '싸이코' '실질' '실은' '실수'
 '실사' '실망' '치즈' '치킨' '실패' '치히로' '친구' '신하균' '친다' '신파' '신선' '신부' '신발' '칙칙'
 '실패작' '실화' '심각' '싸이' '취한다' '싸움' '싸우' '취향' '싸구려' '싱크로' '싱겁' '심형래' '심했'
 '심한데' '심하' '심장' '심심풀이' '치밀' '심심' '심리' '취지' '0개']


In [22]:
print(grid.best_estimator_.named_steps["logisticregression"])

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


# 카카오 형태소 분석기 기준

성능이 더 떨어짐. 여러가지 이유가 있겠지만 사용자 정의 사전 미구축 및 은어 등의 단어를 인식을 못하는 문제일 가능성이 높다.

In [23]:
data = pd.read_csv('./data_after_tokenizer_kakao/ratings_train_after_prepro.txt', header=None)
data_test = pd.read_csv('./data_after_tokenizer_kakao/ratings_test_after_prepro.txt', header=None)

In [24]:
df = data.sample(frac=1).reset_index(drop=True)
print(df.iloc[0:10,2])

0    0
1    0
2    0
3    1
4    0
5    1
6    0
7    1
8    1
9    0
Name: 2, dtype: int64


In [25]:
df = df.dropna(how='any')
data_test = data_test.dropna(how='any')

In [26]:
X_train = df.iloc[:, 1].values
y_train = df.iloc[:, 2].values
X_test = data_test.iloc[:, 1].values
y_test = data_test.iloc[:, 2].values

In [27]:
max_word = 32000
max_len = 35

tok = Tokenizer(num_words = max_word)
tok.fit_on_texts(X_train)
print(len(tok.word_index))
#print(tok.word_index)

34388


In [28]:
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [29]:
sequences_test = tok.texts_to_sequences(X_test)
sequences_test_matrix = sequence.pad_sequences(sequences_test, maxlen=max_len)

In [30]:
with K.tf_ops.device('device:GPU:0'):
    model = Sequential()
    #max_words를 50차원에. 즉 20000개의 단어를 50차원에다가. 문장의 길이는 max_len
    model.add(Embedding(max_word, 50, input_length=max_len))
    model.add(LSTM(64))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    model_dir = './model'
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    model_path = model_dir + "/review_lstm_soojin.model"
    
    checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [31]:
hist = model.fit(sequences_matrix, y_train, batch_size=128, epochs=15, validation_split=0.2, callbacks=[checkpoint, early_stopping])

Train on 119996 samples, validate on 29999 samples
Epoch 1/15
119996/119996 [==============================] - 61s 505us/step - loss: 0.5612 - acc: 0.6961 - val_loss: 0.5305 - val_acc: 0.7175

Epoch 00001: val_loss improved from inf to 0.53049, saving model to ./model/review_lstm_soojin.model
Epoch 2/15
119996/119996 [==============================] - 59s 489us/step - loss: 0.4865 - acc: 0.7620 - val_loss: 0.5324 - val_acc: 0.7183

Epoch 00002: val_loss did not improve from 0.53049
Epoch 3/15
119996/119996 [==============================] - 58s 484us/step - loss: 0.4489 - acc: 0.7785 - val_loss: 0.5504 - val_acc: 0.7172

Epoch 00003: val_loss did not improve from 0.53049
Epoch 4/15
119996/119996 [==============================] - 58s 486us/step - loss: 0.4270 - acc: 0.7872 - val_loss: 0.5546 - val_acc: 0.7211

Epoch 00004: val_loss did not improve from 0.53049


In [32]:
print("정확도 : %.4f" % (model.evaluate(sequences_test_matrix, y_test)[1]))

49997/49997 [==============================] - 35s 702us/step
정확도 : 0.7188
